In [1]:
import os
from pathlib import Path
from operator import itemgetter
import pickle
import pydicom
from time import time

import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon, MultiPolygon, LineString, GeometryCollection, Point, MultiPoint
from shapely.affinity import translate

from catch_converter.parse_contours import parse_cvi42ws
from LazyLuna.Mini_LL import *
from LazyLuna.CATCH_utils import *

In [2]:
# basepaths
bp       = '/Users/dietrichhadler/Desktop/Daten/CS_ESED_Cases'
bp_annos = '/Users/dietrichhadler/Desktop/Daten/CS_ESED_Cases/Preds/MRUNet'
bp_cases = '/Users/dietrichhadler/Desktop/Daten/CS_ESED_Cases/Cases'
bp_imgs  = '/Users/dietrichhadler/Desktop/Daten/CS_ESED_Cases/Imgs'

imgsanno_paths = get_imgs_and_annotation_paths(bp_imgs, bp_annos)

cases = []
sax_cs_view   = SAX_CS_View()
for imgp, annop in imgsanno_paths:
    print(imgp, annop)
    st = time()
    case = Case(imgp, annop, os.path.basename(imgp), os.path.basename(bp_annos))
    case = sax_cs_view.customize_case(case)
    case.store(bp_cases)
    cases.append(case)
    print('Case customization took: ', time()-st, 'Case: ', case.reader_name, case.case_name)

/Users/dietrichhadler/Desktop/Daten/CS_ESED_Cases/Imgs/ECSPRESS-012 /Users/dietrichhadler/Desktop/Daten/CS_ESED_Cases/Preds/MRUNet/1.2.276.0.7230010.3.1.2.0.458.1575883615.363792
Case customization took:  10.585137844085693 Case:  MRUNet ECSPRESS-012
/Users/dietrichhadler/Desktop/Daten/CS_ESED_Cases/Imgs/ECSPRESS-003 /Users/dietrichhadler/Desktop/Daten/CS_ESED_Cases/Preds/MRUNet/1.2.276.0.7230010.3.1.2.0.458.1575883432.347603
Case customization took:  10.76145315170288 Case:  MRUNet ECSPRESS-003
/Users/dietrichhadler/Desktop/Daten/CS_ESED_Cases/Imgs/ECSPRESS-056 /Users/dietrichhadler/Desktop/Daten/CS_ESED_Cases/Preds/MRUNet/1.2.276.0.7230010.3.1.2.0.458.1575884610.449611
Case customization took:  11.820950031280518 Case:  MRUNet ECSPRESS-056


In [3]:
sax_cs_view   = SAX_CS_View()
case = cases[0]
print('CS data:')
case = sax_cs_view.customize_case(case)
cat = [cat for cat in case.categories][0]
print('LV endo volume curve: ', cat.get_volume_curve('rv_endo'))
print([cat.phase for cat in case.categories])
for cr in case.crs: print(cr.name, cr.get_cr())
print([cr.get_cr(string=True) for cr in case.crs])

CS data:
LV endo volume curve:  [123.32962374880924, 120.64196955597905, 103.46172346161754, 92.27900794546878, 78.46913206076574, 71.57283609331074, 61.52221928861136, 58.91234286985023, 57.27036763950378, 61.859256309577226, 65.96419438544328, 70.67407070406853, 75.49629269634907, 85.43456382739316, 91.00863763567443, 96.11604479954144, 99.60740265775172, 102.21727907651288, 104.16172342823891, 105.42345176313664, 107.39382203955238, 113.45184644204103, 119.05184617501197, 125.18764835156966, 123.4419694224645]
[9, 24, 8, 23]
LVESV 70.94197192586189
LVEDV 149.5493755849726
RVESV 57.27036763950378
RVEDV 125.18764835156966
LVSV 78.60740365911072
LVEF 52.56284310924526
RVSV 67.91728071206589
RVEF 54.25238161019563
LVMYOMASS 102.97800723932173
RVMYOMASS 0.0
LVESP 9
RVESP 8
LVEDP 24
RVEDP 23
LVEDP 16
['70.94', '149.55', '57.27', '125.19', '78.61', '52.56', '67.92', '54.25', '102.98', '0.00', '9', '8', '24', '23', '16']


In [4]:
case_paths = [p for p in os.listdir(bp_cases) if 'Annos' in p and p.endswith('.pickle')]
cases1 = sorted([pickle.load(open(os.path.join(bp_cases, p), 'rb')) for p in case_paths], key=lambda c:c.case_name)
case_paths = [p for p in os.listdir(bp_cases) if 'UNet' in p and p.endswith('.pickle')]
cases2 = sorted([pickle.load(open(os.path.join(bp_cases, p), 'rb')) for p in case_paths], key=lambda c:c.case_name)

In [6]:
for c1,c2 in zip(cases1, cases2):
    print(c1.case_name, c2.case_name)
    for cr1, cr2 in zip(c1.crs, c2.crs):
        #print(cr1.name, cr1.get_cr_diff(cr2, True))
        print(cr1.name, cr1.name==cr2.name, cr1.get_cr(True), cr2.get_cr(True), cr1.get_cr_diff(cr2, True))
    print()

ECSPRESS-003 ECSPRESS-003
LVESV True 44.67 43.62 1.05
LVEDV True 142.08 137.69 4.39
RVESV True 78.06 62.91 15.15
RVEDV True 156.56 141.02 15.54
LVSV True 97.41 94.07 3.34
LVEF True 68.56 68.32 0.24
RVSV True 78.50 78.11 0.39
RVEF True 50.14 55.39 -5.25
LVMYOMASS True 90.11 83.44 6.67
RVMYOMASS True 0.00 0.00 0.00
LVESP True 10 10 0.00
RVESP True 10 11 1.00
LVEDP True 24 24 0.00
RVEDP True 24 24 0.00
LVEDP True 16 16 0.00

ECSPRESS-012 ECSPRESS-003
LVESV True 78.55 49.39 29.16
LVEDV True 157.61 143.16 14.45
RVESV True 76.38 65.60 10.78
RVEDV True 149.84 143.54 6.30
LVSV True 79.06 93.77 -14.71
LVEF True 50.16 65.50 -15.34
RVSV True 73.46 77.94 -4.48
RVEF True 49.02 54.30 -5.27
LVMYOMASS True 122.17 90.61 31.57
RVMYOMASS True 0.00 0.00 0.00
LVESP True 8 10 2.00
RVESP True 8 9 1.00
LVEDP True 24 24 0.00
RVEDP True 24 24 0.00
LVEDP True 16 16 0.00

ECSPRESS-056 ECSPRESS-012
LVESV True 135.39 70.94 64.45
LVEDV True 244.77 149.55 95.22
RVESV True 86.95 57.27 29.68
RVEDV True 185.49 125.19 60